In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

#### Channel network training and figures

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from narsil2.segmentation.datasets import mmDataMultiSpecies
from narsil2.segmentation.transformations import changedtoPIL, randomCrop, randomRotation, randomAffine, normalize, toTensor, addnoise, padTo16
from narsil2.segmentation.transformations import randomVerticalFlip, shrinkSize
from torchvision import transforms
from pathlib import Path
%matplotlib qt

In [3]:
ROOT_DIR = '/mnt/sda2/Praneeth/ASTFISH/'

DATA_DIR = ROOT_DIR + 'data/EXP-22-BP0394 Analysis code data and models/data/channelsData'

SAVE_DIR = '../../figures/supplementary/'

species = ['channels']

validationTransforms = transforms.Compose([changedtoPIL(), padTo16(),
                             toTensor(), normalize()])

In [4]:
train_dataset = mmDataMultiSpecies(DATA_DIR, species, None, datasetType='validation', 
                                  includeWeights=False)

In [5]:
len(train_dataset)

520

In [6]:
train_dataset[0]

{'phase': array([[3303, 3623, 3517, ..., 2528, 2400, 2461],
        [3622, 3447, 3554, ..., 2527, 2443, 2466],
        [3494, 3569, 3528, ..., 2432, 2470, 2459],
        ...,
        [2989, 3077, 3201, ..., 2304, 2286, 2343],
        [2977, 3178, 3122, ..., 2361, 2402, 2370],
        [3146, 3079, 3021, ..., 2351, 2423, 2393]], dtype=uint16),
 'mask': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 'weights': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'phaseFilename': PosixPath('/mnt/sda2/Praneeth/ASTFISH/data/EXP-22-BP0394 Analysis code data and models/data/channelsData/channels/phase_validation/raw0860.tif')

### Fig 3a

In [7]:
fig, ax = plt.subplots(nrows=1, ncols=1)
plt.tight_layout()
ax.imshow(train_dataset[0]['phase'][300:600, 300:600], cmap='gray')
ax.set_axis_off()
save_filename = SAVE_DIR + 'fig3/phase_3a.svg'
fig.savefig(save_filename, bbox_inches='tight', pad_inches=0)
plt.close(fig)

fig, ax = plt.subplots(nrows=1, ncols=1)
plt.tight_layout()
ax.imshow(train_dataset[0]['mask'][300:600, 300:600], cmap='gray')
ax.set_axis_off()
save_filename = SAVE_DIR + 'fig3/mask_3a.svg'
fig.savefig(save_filename, bbox_inches='tight', pad_inches=0)
plt.close(fig)

### Fig 3b

To get the convergence of the network during training.

Run the script train_channel_net.py in the narsil2.segmentation directory

### Fig 3c

In [8]:
from narsil2.segmentation.model_dev_old import testNet
from narsil2.segmentation.datasets import phaseTestDir
from narsil2.segmentation.transformations import UnetTestTransformations
import torch
from narsil2.segmentation.networks import smallerUnet
from skimage import measure
from skimage.morphology import binary_dilation, remove_small_objects, remove_small_holes
from scipy.signal import find_peaks

In [9]:
channel_model_path = '../../saved_models/channels.pth'

phase_dir = ROOT_DIR + 'data/EXP-22-BV6196 AST FISH 220214/The rule/Pos118/phase/'
save_dir = None
fileformat = '.tiff'

imgTransforms = UnetTestTransformations(pad_to=16, add_noise=True, return_tensors=True) 


In [10]:
test_dataset = phaseTestDir(phase_dir, transform = imgTransforms)

In [11]:
test_dataset[0]

{'phase': tensor([[[-3.3779e-01, -3.0791e-01, -3.6764e-01,  ..., -4.6960e-01,
           -3.4081e-01, -5.3440e-01],
          [-3.9807e-01, -6.6639e-01, -6.6544e-02,  ..., -3.5483e-01,
           -3.9748e-01, -6.0679e-02],
          [-3.0823e-01, -4.4295e-01, -4.8572e-01,  ..., -2.6598e-01,
           -3.9355e-01, -2.6569e-01],
          ...,
          [-1.5419e-02, -8.0815e-02, -5.6319e-04,  ...,  1.9354e-02,
           -1.1157e-01,  1.4320e-02],
          [-9.3375e-02, -3.5232e-02,  1.7443e-01,  ...,  1.2423e-01,
            9.3997e-02, -1.7748e-01],
          [ 1.8587e-01, -2.1337e-02, -4.1403e-01,  ...,  1.0323e-01,
           -2.1943e-01, -1.5926e-01]]]),
 'filename': '/mnt/sda2/Praneeth/ASTFISH/data/EXP-22-BV6196 AST FISH 220214/The rule/Pos118/phase/img_000000000.tiff',
 'raw_shape': (1956, 4080)}

In [12]:
test_dataset[0]['phase'].shape

torch.Size([1, 1968, 4080])

#### Run through the model and find contours

In [13]:
saved_model = torch.load(channel_model_path)
net = smallerUnet(saved_model['modelParameters']['transposeConv'])

net.load_state_dict(saved_model['model_state_dict'])
net.eval()
contours_threshold=0.80

save_filename = SAVE_DIR + 'fig3/overlay_3c.png'
with torch.no_grad():
    for i_batch, datapoint in enumerate(test_dataset, 0):
        phase_img = datapoint['phase'].unsqueeze(0)
        
        mask_pred = net(phase_img)
        
        mask_pred = torch.sigmoid(mask_pred)
        
        mask_pred_cpu = mask_pred.numpy().squeeze(0).squeeze(0) > 0.85
        mask_pred_cpu = remove_small_objects(mask_pred_cpu, min_size=500)
        phase_cpu = phase_img.numpy().squeeze(0).squeeze(0)
        print(mask_pred.shape)
        
        # plotting contuours
        contours = measure.find_contours(mask_pred_cpu, contours_threshold)
        fig, ax = plt.subplots()
        ax.imshow(phase_cpu, cmap='gray')
        ax.set_axis_off()
        for contour in contours:
            ax.plot(contour[:, 1], contour[:, 0], linewidth=1, color='r')
        
        plt.show(block=False)
        fig.savefig(save_filename, bbox_inches='tight', pad_inches=0)
        if i_batch == 0:
            break

torch.Size([1, 1, 1968, 4080])


#### Fig 3d

#### Plot the distribution of channels on data not seen by the network

In [14]:
validation_data_dir = ROOT_DIR + 'data/EXP-22-BP0394 Analysis code data and models/data/channelsData/'
fileformat = '*.tif'

In [15]:
test_dataset = mmDataMultiSpecies(validation_data_dir, species=['channels'],
                                  datasetType='validation', includeWeights=False,
                                  transforms = None)

In [16]:
len(test_dataset)

520

In [17]:
test_dataset[0]

{'phase': array([[3303, 3623, 3517, ..., 2528, 2400, 2461],
        [3622, 3447, 3554, ..., 2527, 2443, 2466],
        [3494, 3569, 3528, ..., 2432, 2470, 2459],
        ...,
        [2989, 3077, 3201, ..., 2304, 2286, 2343],
        [2977, 3178, 3122, ..., 2361, 2402, 2370],
        [3146, 3079, 3021, ..., 2351, 2423, 2393]], dtype=uint16),
 'mask': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 'weights': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'phaseFilename': PosixPath('/mnt/sda2/Praneeth/ASTFISH/data/EXP-22-BP0394 Analysis code data and models/data/channelsData/channels/phase_validation/raw0860.tif')

In [18]:
channel_transformations = UnetTestTransformations()

In [19]:
def construct_sample(phase_img, filename):
    channel_transformations = UnetTestTransformations(pad_to=16, add_noise=True, return_tensors=True)
    sample = {
        'phase': phase_img,
        'filename': filename, 
        'raw_shape': phase_img.shape
    }
    return channel_transformations(sample)['phase'].unsqueeze(0)
    

In [20]:
phase_tensor = construct_sample(test_dataset[0]['phase'], test_dataset[0]['filename'])

In [21]:
phase_tensor.shape

torch.Size([1, 1, 880, 1024])

In [22]:
def predict_mask(phase_tensor):
    with torch.no_grad():
        mask_pred = net(phase_tensor)
        mask_pred = torch.sigmoid(mask_pred)
        mask_pred_cpu = mask_pred.numpy().squeeze(0).squeeze(0) > 0.9
    
    return mask_pred_cpu

In [23]:
# This slightly different from t
params = {
    'channel_min':200, 
    'channel_max': 800,
    'channel_sum': 200,
    'hist_peaks_distance': 30,
    
}

In [24]:
def get_locations(mask, params):
    hist = np.sum(mask[params['channel_min']: params['channel_max'], :], axis=0) > params['channel_sum']
    peaks, _ = find_peaks(hist, distance=params['hist_peaks_distance'])
    return peaks

In [25]:
get_locations(test_dataset[0]['mask'], params)

array([ 13,  66, 119, 172, 225, 278, 331, 384, 438, 490, 543, 597, 651,
       703, 757, 810, 916, 971])

In [26]:
mask_pred = predict_mask(phase_tensor)

In [27]:
plt.imshow(mask_pred)

In [28]:
get_locations(mask_pred, params)

array([ 13,  66, 119, 172, 225, 278, 331, 384, 437, 490, 543, 597, 650,
       703, 757, 810, 916, 970])

In [29]:
diffs = []
for i in range(len(test_dataset)):
    datapoint = test_dataset[i]
    phase_tensor = construct_sample(datapoint['phase'], datapoint['filename'])
    mask_pred = predict_mask(phase_tensor)
    pred_locations = get_locations(mask_pred, params)
    true_locations = get_locations(datapoint['mask'], params)
    print(i, len(pred_locations), " ---- ", len(true_locations), datapoint['filename'])
    print(pred_locations)
    print(true_locations)
    if len(pred_locations) == len(true_locations):
        diffs.extend(list(pred_locations - true_locations))
    

0 18  ----  18 raw0860.tif
[ 13  66 119 172 225 278 331 384 437 490 544 597 650 703 757 810 916 970]
[ 13  66 119 172 225 278 331 384 438 490 543 597 651 703 757 810 916 971]
1 18  ----  18 raw0835.tif
[ 12  65 118 171 224 277 330 383 436 489 543 596 649 702 755 809 915 969]
[ 12  65 118 171 224 277 330 384 436 490 543 596 649 702 756 810 916 969]
2 17  ----  17 raw0506.tif
[ 25 131 184 238 291 345 398 451 504 558 611 664 717 771 824 877 930]
[ 24 131 185 237 291 345 399 451 505 558 611 664 717 771 823 876 930]
3 18  ----  18 img0501.tif
[ 79 132 185 239 291 345 398 451 504 557 611 664 717 771 825 878 931 986]
[ 80 133 186 239 292 345 398 451 503 558 611 665 718 771 825 878 932 986]
4 17  ----  17 raw0060.tif
[ 19 125 179 232 286 339 392 446 499 552 605 658 712 765 818 871 924]
[ 19 124 178 232 285 339 393 445 499 552 605 659 711 765 818 871 924]
5 18  ----  17 img0007.tif
[ 70 123 176 229 282 335 388 441 494 548 601 654 708 761 814 868 921 976]
[ 70 175 229 283 335 388 441 494 548 602

48 17  ----  17 raw1779.tif
[ 20 127 181 234 287 341 394 447 501 554 607 660 713 767 820 873 926]
[ 21 127 181 234 288 341 394 447 501 554 607 660 714 767 820 873 926]
49 18  ----  18 img0132.tif
[ 75 129 182 235 288 341 394 447 500 553 607 660 713 767 820 874 927 982]
[ 75 129 181 235 287 341 394 447 500 553 606 659 713 767 820 874 927 982]
50 18  ----  18 raw0536.tif
[  9  62 115 168 221 274 327 380 433 487 540 593 646 699 753 806 912 966]
[ 10  62 115 168 221 274 328 381 434 487 540 593 646 700 754 807 913 966]
51 17  ----  17 raw0171.tif
[ 25 131 184 238 291 345 398 451 505 558 611 664 717 770 824 877 930]
[ 25 131 185 238 291 346 398 452 505 558 611 664 718 770 823 878 930]
52 17  ----  17 raw0534.tif
[ 62 115 168 221 274 327 380 433 486 539 592 646 699 752 806 912 965]
[ 62 115 168 221 274 327 380 434 486 539 593 646 699 753 806 912 965]
53 18  ----  17 raw1174.tif
[  19  125  178  232  285  339  392  445  498  552  605  658  711  765
  818  871  924 1022]
[ 19 124 178 232 285 33

95 18  ----  18 raw0831.tif
[ 12  65 118 171 224 278 330 384 437 490 543 596 649 703 756 810 916 969]
[ 12  66 118 171 225 277 331 383 436 490 543 596 650 703 756 810 916 969]
96 18  ----  18 raw1827.tif
[  56  109  162  215  268  321  374  427  480  534  587  640  693  746
  800  906  960 1013]
[  56  109  162  216  269  322  374  428  481  534  587  640  694  746
  800  907  960 1013]
97 18  ----  18 raw1013.tif
[ 11  64 117 170 223 276 329 382 435 488 541 594 648 701 754 808 914 967]
[ 10  64 117 170 223 276 329 382 435 489 541 594 648 701 754 808 915 967]
98 18  ----  18 raw0985.tif
[  9  63 116 169 222 275 327 381 434 487 540 593 646 699 753 807 912 966]
[ 10  62 116 168 222 275 328 381 434 487 540 593 646 700 753 806 912 967]
99 18  ----  18 raw1740.tif
[ 14  67 120 173 226 279 332 385 438 491 544 598 651 704 758 811 917 971]
[ 14  67 120 173 226 279 332 385 438 491 545 598 651 704 758 812 917 971]
100 18  ----  18 img0099.tif
[ 72 126 179 232 285 338 391 444 497 550 604 657 710 

142 17  ----  17 raw0385.tif
[ 61 114 167 220 273 326 379 432 486 539 592 645 698 752 806 911 965]
[ 61 114 167 221 273 327 380 433 486 538 592 645 698 752 806 912 965]
143 17  ----  17 raw0525.tif
[ 61 114 167 220 273 326 379 432 485 538 591 645 698 751 805 911 965]
[ 61 114 167 220 273 326 380 432 485 539 592 645 699 752 806 911 965]
144 17  ----  17 raw0472.tif
[ 22 129 182 235 289 342 395 449 502 555 609 662 715 768 821 874 928]
[ 22 129 182 235 289 342 395 449 502 555 609 662 715 768 821 874 928]
145 17  ----  17 raw1314.tif
[ 20 126 180 233 287 340 393 447 500 553 607 660 713 766 819 872 925]
[ 20 127 180 233 287 341 393 446 500 553 606 660 712 766 819 872 925]
146 17  ----  17 raw1683.tif
[ 60 113 166 219 272 325 378 431 485 538 591 644 697 751 804 910 964]
[ 60 113 166 219 272 325 379 432 484 538 590 644 697 750 804 910 964]
147 18  ----  18 raw0392.tif
[  9  63 116 169 222 275 328 381 434 487 540 593 647 700 753 807 913 966]
[ 10  63 116 169 222 275 328 380 434 487 540 593 647

189 17  ----  17 raw1092.tif
[ 27 133 187 241 294 347 400 454 507 560 614 667 720 773 826 879 933]
[ 27 133 187 241 294 347 401 453 507 561 613 667 720 773 826 879 932]
190 17  ----  17 raw0144.tif
[ 59 112 165 218 271 324 377 430 483 536 590 643 696 749 803 909 963]
[ 59 112 165 218 271 324 377 431 483 536 589 644 696 749 804 909 963]
191 17  ----  17 raw0942.tif
[ 25 130 184 238 291 344 398 451 504 558 611 664 717 770 823 876 930]
[ 24 130 185 238 291 345 399 451 504 558 611 664 717 771 824 876 930]
192 17  ----  17 raw0374.tif
[ 60 114 167 219 272 326 378 432 485 538 591 644 697 751 804 910 964]
[ 60 114 167 220 273 326 379 432 485 538 591 644 698 751 805 910 964]
193 17  ----  17 raw1778.tif
[ 20 126 180 233 286 340 393 447 500 553 606 660 713 766 819 872 926]
[ 20 127 179 233 287 341 393 446 500 554 606 659 713 766 819 872 925]
194 18  ----  17 img0564.tif
[ 76 129 183 236 289 342 395 448 501 555 608 661 715 768 822 875 929 983]
[ 76 130 183 236 289 343 395 448 501 554 608 661 716

...

330 17  ----  17 raw1488.tif
[ 27 133 186 240 293 346 400 453 507 560 613 666 719 772 825 879 932]
[ 26 132 186 239 293 346 400 453 507 560 613 666 718 772 825 879 932]
331 18  ----  18 raw1126.tif
[ 11  64 118 170 224 277 329 382 435 488 542 595 648 702 755 808 914 968]
[ 11  65 117 170 224 276 330 383 436 489 542 595 649 702 756 808 915 968]
332 19  ----  18 img0491.tif
[  80  134  187  240  293  346  399  452  505  559  612  665  719  772
  826  879  933  987 1039]
[ 81 135 187 240 293 347 399 452 505 559 611 666 719 772 826 879 933 987]
333 17  ----  17 raw1815.tif
[ 22 128 181 235 288 342 395 448 502 555 608 661 714 767 821 874 927]
[ 22 128 181 235 288 343 395 449 501 555 608 661 715 767 821 873 927]
334 17  ----  17 raw0486.tif
[ 24 130 184 237 290 344 397 450 504 557 610 664 717 770 823 876 929]
[ 24 130 183 237 291 344 397 451 503 556 611 663 717 769 823 876 929]
335 18  ----  18 img0641.tif
[ 78 131 184 238 291 344 397 450 503 556 610 663 717 770 824 877 930 984]
[ 78 133 185

377 18  ----  18 img0207.tif
[ 71 124 177 230 283 336 389 442 496 549 602 655 708 762 815 869 923 977]
[ 70 124 176 229 283 336 389 442 495 549 602 654 708 762 815 869 922 976]
378 18  ----  18 raw1558.tif
[ 13  66 119 172 225 278 331 384 437 490 543 596 650 703 756 810 916 969]
[ 12  66 119 172 225 278 331 384 437 491 543 596 650 703 756 810 916 970]
379 18  ----  18 raw1556.tif
[ 13  66 119 172 225 278 331 384 438 490 544 597 650 703 757 810 916 970]
[ 13  66 120 172 226 278 332 384 438 491 544 597 650 703 757 810 916 971]
380 18  ----  18 raw1595.tif
[ 15  68 121 174 227 280 333 386 439 493 546 599 652 705 759 812 918 972]
[ 15  68 121 175 227 281 334 387 440 492 546 599 652 705 759 812 919 971]
381 18  ----  18 raw0994.tif
[ 10  63 116 169 222 275 328 381 434 487 540 594 647 700 753 807 913 967]
[ 10  63 116 170 223 275 328 382 435 488 541 593 647 700 753 807 914 967]
382 17  ----  17 raw1670.tif
[ 27 133 187 240 294 347 400 454 507 560 614 667 720 773 826 879 933]
[ 27 133 186 240

424 17  ----  17 raw0457.tif
[ 21 127 181 234 288 341 394 448 501 554 607 661 714 767 820 873 926]
[ 21 127 181 235 287 341 395 448 501 554 607 661 714 767 820 873 926]
425 18  ----  18 img0395.tif
[ 73 126 179 232 285 338 392 445 498 551 604 658 711 764 818 872 925 979]
[ 73 127 179 233 285 338 392 445 498 551 605 658 711 764 818 871 925 979]
426 18  ----  18 raw0266.tif
[ 14  67 120 173 226 279 332 385 438 491 544 598 651 704 758 811 917 971]
[ 14  67 120 174 226 279 332 386 438 492 545 598 652 704 757 812 917 970]
427 17  ----  17 raw0926.tif
[ 23 129 183 236 289 343 396 449 503 556 609 662 715 769 822 875 928]
[ 23 129 183 236 290 343 396 450 503 556 609 662 716 769 822 875 928]
428 18  ----  18 img0506.tif
[ 79 133 186 239 292 345 398 451 504 558 611 664 718 771 825 878 931 986]
[ 80 133 186 239 293 345 399 451 505 558 611 665 718 772 825 879 932 986]
429 18  ----  18 raw0088.tif
[  55  108  161  215  268  320  374  426  480  533  586  639  692  746
  799  906  959 1012]
[  56  10

471 18  ----  18 raw0858.tif
[ 13  66 119 172 225 278 331 384 437 490 544 597 650 703 757 810 916 970]
[ 13  67 119 172 225 279 331 385 438 491 544 597 651 704 757 811 918 971]
472 18  ----  18 raw0229.tif
[ 10  63 117 169 223 275 328 382 435 488 541 594 647 701 754 808 914 967]
[ 10  64 116 170 222 276 328 381 435 488 541 594 648 701 754 808 914 968]
473 18  ----  18 img0165.tif
[ 74 127 180 233 286 339 393 446 499 552 605 659 712 765 819 872 926 980]
[ 75 128 181 233 287 340 393 446 499 552 605 659 713 766 819 873 927 981]
474 19  ----  18 img0414.tif
[  78  131  184  237  290  344  397  450  503  556  609  663  716  770
  823  877  930  984 1039]
[ 78 132 185 238 291 344 397 450 504 557 609 662 716 769 824 876 930 985]
475 18  ----  18 img0325.tif
[ 76 129 182 235 288 341 394 447 500 554 607 661 714 767 821 874 928 982]
[ 75 129 182 235 289 341 395 447 501 554 606 661 714 767 820 875 928 982]
476 18  ----  18 raw0407.tif
[ 10  63 116 169 222 275 328 381 434 487 541 594 647 700 754 8

518 17  ----  17 raw1665.tif
[ 27 133 187 241 294 347 401 454 507 561 614 667 720 773 826 879 933]
[ 27 134 187 241 295 347 401 454 507 561 614 667 720 773 827 879 933]
519 17  ----  17 raw1226.tif
[ 23 129 183 236 290 343 396 450 503 556 609 663 716 769 822 875 928]
[ 23 129 182 236 289 343 396 450 503 557 609 663 715 770 822 875 928]


In [30]:
fig, ax = plt.subplots()
ax.bar(*np.unique(diffs, return_counts=True))
ax.set_ylabel('Number of channels')
ax.set_xlabel('Pixel offset between predicted and true channel locations',)
save_filename = SAVE_DIR + 'fig3/pixel_offsets.svg'
fig.savefig(save_filename, bbox_inches='tight', pad_inches=0)

In [31]:
np.unique(diffs, return_counts=True)

(array([-3, -2, -1,  0,  1,  2,  3]),
 array([   3,   22, 1610, 5351,  712,    8,    1]))